In [1]:
import tensorflow as tf
import numpy as np
from transformer_video import VideoPrediction
import matplotlib.pyplot as plt

def generate_movies(n_samples=1200, n_frames=20):
    row = 80
    col = 80
    noisy_movies = np.zeros((n_samples, n_frames, row, col, 1), dtype=float)
    shifted_movies = np.zeros((n_samples, n_frames, row, col, 1),
                              dtype=float)

    for i in range(n_samples):
        # Add 3 to 7 moving squares
        n = np.random.randint(3, 8)

        for j in range(n):
            # Initial position
            xstart = np.random.randint(20, 60)
            ystart = np.random.randint(20, 60)
            # Direction of motion
            directionx = np.random.randint(0, 3) - 1
            directiony = np.random.randint(0, 3) - 1

            # Size of the square
            w = np.random.randint(2, 4)

            for t in range(n_frames):
                x_shift = xstart + directionx * t
                y_shift = ystart + directiony * t

                # Shift the ground truth by 1
                x_shift = xstart + directionx * (t + 1)
                y_shift = ystart + directiony * (t + 1)
                shifted_movies[i, t, x_shift - w: x_shift + w,
                               y_shift - w: y_shift + w, 0] += 1

    # Cut to a 40x40 window
    shifted_movies = shifted_movies[::, ::, 20:60, 20:60, ::]
    shifted_movies[shifted_movies >= 1] = 1
    return shifted_movies
    
    
def load_dataset(path, filename):
    train_data = np.load(path + filename)
    train_data = train_data.swapaxes(0, 1)[:100]
    # train_data[[1005, 9000]] = train_data[[9000, 1005]]

    # patch size 2 x 2
    # train_data = train_data.reshape(train_data.shape[0], train_data.shape[1], 16, 16, 16)
    #train_data = reshape_patch(train_data, (2, 2))
    #plt.imshow(restore_patch(train_data[0], (2, 2))[0])
    #plt.show()
    
    train_data[train_data < 128] = 0
    train_data[train_data >= 128] = 1
    #train_data = train_data / 255.0
    print(train_data.shape)
    
    train_data = np.expand_dims(train_data, axis=4)
    X = train_data[:, :10, :, :, :]
    Y = train_data[:, 10:21, :, :, :]
    

    X = tf.convert_to_tensor(X, dtype=tf.float32)
    Y = tf.convert_to_tensor(Y, dtype=tf.float32)
    
    return (X, Y)
    
def plot_result(input_, actual, predict):
    
        
    for i in range(actual.shape[0]):
        plt.subplot(121), plt.imshow(actual[i]),
        plt.title("Actual_" + str(i + 1 + input_.shape[0]))
        plt.subplot(122), plt.imshow(predict[i]),
        plt.title("Predicted_" + str(i + 1 + input_.shape[0]))
        plt.show()
    

2023-03-15 09:18:53.511238: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-15 09:18:53.544463: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
dataset = np.load('/home/faster/Documents/wind_prediction/10m_u-component_of_wind_2022_hourly.npz')['arr_0']


In [3]:
dataset = np.expand_dims(dataset, axis=-1)
dataset = (dataset - dataset.min(axis=0)) / (dataset.max(axis=0) - dataset.min(axis=0))

In [4]:
dataset = dataset[:,:,:,:,:]

In [5]:
dataset.shape

(730, 12, 201, 201, 1)

In [6]:

#shifted_movies = tf.convert_to_tensor(generate_movies(n_samples=1200), dtype=tf.float32)
#print(shifted_movies.shape)

In [7]:

#X = shifted_movies[:, :10, :, :, :]
#Y = shifted_movies[:, 10:, :, :, :]
X = dataset[:, :6, :, :, :]
Y = dataset[:, 6:, :, :, :]

# defines the model
model = VideoPrediction(
    num_layers=2, d_model=64, num_heads=16, dff=128,
    filter_size=(5, 5), image_shape=(201, 201), pe_input=10,
    pe_target=20, out_channel=1, loss_function='bin_cross'
)

# training on first 1000 samples
# samples from 1000 - 1199 are used as test set



2023-03-15 09:19:00.975195: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-15 09:19:00.987687: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-15 09:19:00.988209: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-15 09:19:00.988964: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the

In [8]:
model.train(X[:700, :3], X[:700, 3:], None, None, 10, 1,epoch_print=1)

2023-03-15 09:19:02.456558: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401
2023-03-15 09:19:03.053606: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-15 09:19:03.053893: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-15 09:19:03.053905: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-03-15 09:19:03.054210: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-15 09:19:03.054231: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


Epoch 1 Loss 0.6638
Time taken for 1 epoch 167.30358695983887 sec

Epoch 2 Loss 0.6388
Time taken for 1 epoch 162.16580319404602 sec

Epoch 3 Loss 0.6299
Time taken for 1 epoch 162.24408316612244 sec

Epoch 4 Loss 0.6301
Time taken for 1 epoch 162.37160325050354 sec

Epoch 5 Loss 0.6258
Time taken for 1 epoch 161.6639211177826 sec

Epoch 6 Loss 0.6247
Time taken for 1 epoch 160.56168723106384 sec

Epoch 7 Loss 0.6244
Time taken for 1 epoch 160.68697571754456 sec

Epoch 8 Loss 0.6243
Time taken for 1 epoch 160.8771150112152 sec

Epoch 9 Loss 0.6241
Time taken for 1 epoch 160.06910395622253 sec

Epoch 10 Loss 0.6240
Time taken for 1 epoch 160.93549942970276 sec



In [9]:
model.transformer.save_weights('./2lfil5-5')

In [23]:
test_loss = model.evaluate(X[100:300], Y[100:300], 8)
print('Test Loss {:.4f}'.format(test_loss))

2023-03-14 18:38:42.507909: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 6.25GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


Test Loss 0.7631


In [13]:
test_loss = model.evaluate(X[100:300], Y[100:300], 8)
print('Test Loss {:.4f}'.format(test_loss))

Test Loss 1.7469


In [10]:
test_loss = model.evaluate(X[100:300], Y[100:300], 8)
print('Test Loss {:.4f}'.format(test_loss))

Test Loss 0.8666


In [11]:
import io
import imageio
from IPython.display import Image, display
from ipywidgets import widgets, Layout, HBox


x1 = tf.concat((X[242], Y[242]), axis=0)
x2 = tf.concat((X[414], Y[414]), axis=0)
y1 = model.predict(x1[:6], 6)
y2 = model.predict(x2[:6], 6)
x1 = x1.numpy().reshape(12, 201, 201)
x2 = x2.numpy().reshape(12, 201, 201)

from datetime import datetime
images = []
for i in range(y1.shape[0]):
    images.append((np.squeeze(x1[i]) * 255).astype('uint8'))
imageio.mimsave(str(datetime.now())+'x.gif', images,fps=2)

images = []
for i in range(y1.shape[0]):
    images.append((np.squeeze(y1[i]) * 255).astype('uint8'))
imageio.mimsave(str(datetime.now())+'y.gif', images,fps=2)


In [15]:
import io
import imageio
from IPython.display import Image, display
from ipywidgets import widgets, Layout, HBox


x1 = tf.concat((X[111], Y[111]), axis=0)
y1 = model.predict(x1[:6], 20)
x1 = x1.numpy().reshape(12, 201, 201)

from datetime import datetime
images = []
for i in range(x1.shape[0]):
    images.append((np.squeeze(x1[i]) * 255).astype('uint8'))
imageio.mimsave(str(datetime.now())+'x.gif', images,fps=2)

images = []
for i in range(y1.shape[0]):
    images.append((np.squeeze(y1[i]) * 255).astype('uint8'))
imageio.mimsave(str(datetime.now())+'y.gif', images,fps=2)

